In [1]:
# default_exp chat

# get

> API details.

In [2]:
#export
import os, openai
from awsSchema.apigateway import Event, Response
from copy import deepcopy
from beartype import beartype
from nicHelper.secrets import getSecret
from openai import Completion

In [3]:
#export
class ParseInputError(Exception):
    pass

In [4]:
#export
EventInput = dict

@beartype
def parseEvent(event):
    eventCopy = deepcopy(event)
    body = Event.parseBody(eventCopy)

    try:
        inputText = body['inputText']
    except KeyError:
        raise ParseInputError('inputText is not in body')
    
    return inputText

@beartype
def talkWithOpenAI(inputText: str)->str:
    openai.api_key = getSecret('openai')['key']
    r = Completion.create(engine="davinci", prompt=inputText, max_tokens=100, temperature=0.2)
    print("r: ", r)

    return r["choices"][0]["text"]

In [5]:
#export
def chat(event, *args):
    try:
        inputText = parseEvent(event)
        response = talkWithOpenAI(inputText)
        return Response.returnSuccess(body = response)
    except Exception as e:
        return Response.returnError(message=str(e))

In [6]:
#hide
import json
inputSample = {"inputText": "Hello my name is sam. What is your favorite thinga about me?"}
input_ = Event.getInput(inputSample)
print("\n input is", input_, "\n")
x = chat(input_)
print("body = ", x)


 input is {'body': '{"inputText":"Hello my name is sam. What is your favorite thinga about me?"}', 'httpMethod': None, 'multiValueHeaders': None, 'multiValueQueryStringParameters': None, 'path': None, 'pathParameters': None, 'queryStringParameters': {}, 'requestContext': None, 'headers': {}, 'statusCode': 200, 'isBase64Encoded': False} 

body =  {'body': '{"error":"An error occurred (AccessDeniedException) when calling the GetSecretValue operation: User: arn:aws:iam::394922924679:user\\/tenxorSamuelCLIUser is not authorized to perform: secretsmanager:GetSecretValue on resource: openai because no identity-based policy allows the secretsmanager:GetSecretValue action"}', 'statusCode': 400, 'headers': {'Access-Control-Allow-Headers': '*', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': '*'}}


In [ ]:
#hide
